In [ ]:
import random
import os

class MazeGame:
    def __init__(self, size=5):
        self.size = size
        self.player_pos = [0, 0]                 # [row, col]
        self.exit_pos = [size-1, size-1]         # Bottom-right corner
        self.health = 3
        self.visited = set()
        self.game_over = False
        self.won = False

        # ⚠️ Define room types BEFORE generating maze
        self.EMPTY = '.'
        self.SNAKE = 'S'
        self.BOMB = 'B'
        self.TELEPORTER = 'Y'
        self.EXIT = 'X'
        self.PLAYER = 'P'

        self.maze = self.generate_maze()

    def generate_maze(self):
        """Generate a random maze with different room types"""
        maze = [[self.EMPTY for _ in range(self.size)] for _ in range(self.size)]

        # Place exit
        maze[self.exit_pos[0]][self.exit_pos[1]] = self.EXIT

        # Calculate total rooms (excluding start and exit)
        total_rooms = self.size * self.size - 2

        # Distribute room types
        num_snakes = max(1, total_rooms // 4)       # ~25% snakes
        num_bombs = max(1, total_rooms // 6)        # ~17% bombs
        num_teleporters = max(1, total_rooms // 8)  # ~12% teleporters

        # Create list of positions to fill (excluding start and exit)
        positions = []
        for r in range(self.size):
            for c in range(self.size):
                if [r, c] != self.player_pos and [r, c] != self.exit_pos:
                    positions.append([r, c])

        # Randomly assign room types
        random.shuffle(positions)

        # Place snakes
        idx = 0
        for _ in range(min(num_snakes, len(positions))):
            r, c = positions[idx]; idx += 1
            maze[r][c] = self.SNAKE

        # Place bombs
        for _ in range(min(num_bombs, len(positions) - idx)):
            r, c = positions[idx]; idx += 1
            maze[r][c] = self.BOMB

        # Place teleporters
        for _ in range(min(num_teleporters, len(positions) - idx)):
            r, c = positions[idx]; idx += 1
            maze[r][c] = self.TELEPORTER

        return maze

    def clear_screen(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    def print_maze(self):
        print("\n" + "="*50)
        print("🏰 MAZE ESCAPE ADVENTURE 🏰")
        print("="*50)

        for r in range(self.size):
            row_display = ""
            for c in range(self.size):
                if [r, c] == self.player_pos:
                    row_display += "🧙 "
                elif (r, c) in self.visited:
                    room_type = self.maze[r][c]
                    if room_type == self.EMPTY:
                        row_display += "⬜ "
                    elif room_type == self.SNAKE:
                        row_display += "🐍 "
                    elif room_type == self.BOMB:
                        row_display += "💣 "
                    elif room_type == self.TELEPORTER:
                        row_display += "✨ "
                    elif room_type == self.EXIT:
                        row_display += "🚪 "
                elif self.maze[r][c] == self.TELEPORTER:
                    row_display += "⭐ "
                elif [r, c] == self.exit_pos:
                    row_display += "🎯 "
                else:
                    row_display += "❓ "
            print(f"  {row_display}")

        print(f"\n❤️  Health: {self.health}/3")
        print(f"📍 Position: ({self.player_pos[0]}, {self.player_pos[1]})")
        print(f"🎯 Exit: ({self.exit_pos[0]}, {self.exit_pos[1]})")

    def get_room_hint(self, row, col):
        hints = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.size and 0 <= nc < self.size:
                room_type = self.maze[nr][nc]
                if room_type == self.SNAKE and (nr, nc) not in self.visited:
                    hints.append("🐍 You hear hissing nearby...")
                elif room_type == self.BOMB and (nr, nc) not in self.visited:
                    hints.append("💥 You smell gunpowder in the air...")
                elif room_type == self.TELEPORTER and (nr, nc) not in self.visited:
                    hints.append("✨ The air shimmers with magical energy...")
        return hints

    def handle_room_event(self):
        r, c = self.player_pos
        room_type = self.maze[r][c]

        if room_type == self.EMPTY:
            print("🟢 Safe room - You can rest here.")

        elif room_type == self.SNAKE:
            print("🐍 SNAKE ATTACK! A venomous snake bites you!")
            self.health -= 1
            print(f"💔 You lose 1 health. Health remaining: {self.health}")
            if self.health <= 0:
                print("💀 The snake's venom overwhelms you...")
                self.game_over = True

        elif room_type == self.BOMB:
            print("💣 BOOM! You stepped on a bomb!")
            print("💀 The explosion is instant and deadly...")
            self.health = 0
            self.game_over = True

        elif room_type == self.TELEPORTER:
            print("✨ MAGICAL TELEPORTER ACTIVATED!")
            print("🌀 Reality swirls around you as you're transported...")

            # Find a random *safe* destination (not bomb, not exit, not current)
            candidates = []
            for row in range(self.size):
                for col in range(self.size):
                    if self.maze[row][col] != self.BOMB and [row, col] != self.exit_pos and [row, col] != self.player_pos:
                        candidates.append([row, col])

            if candidates:
                new_pos = random.choice(candidates)
                self.player_pos = new_pos
                print(f"⚡ You materialize at position ({new_pos[0]}, {new_pos[1]})!")
                # After teleport, we won't chain-trigger another event immediately.
                # It will happen on the next loop iteration.

        elif room_type == self.EXIT:
            print("🎉 CONGRATULATIONS! You found the exit!")
            print("🏆 You have successfully escaped the maze!")
            self.won = True
            self.game_over = True

    def move_player(self, direction):
        r, c = self.player_pos

        if direction.upper() == 'N':
            new_r, new_c = r - 1, c; dir_name = "North"
        elif direction.upper() == 'S':
            new_r, new_c = r + 1, c; dir_name = "South"
        elif direction.upper() == 'E':
            new_r, new_c = r, c + 1; dir_name = "East"
        elif direction.upper() == 'W':
            new_r, new_c = r, c - 1; dir_name = "West"
        else:
            print("❌ Invalid direction! Use N, S, E, or W")
            return False

        if new_r < 0 or new_r >= self.size or new_c < 0 or new_c >= self.size:
            print(f"🚧 You hit a wall trying to go {dir_name}! Choose another direction.")
            return False

        self.player_pos = [new_r, new_c]
        print(f"🚶 Moving {dir_name}...")
        return True

    def show_hints(self):
        hints = self.get_room_hint(self.player_pos[0], self.player_pos[1])
        if hints:
            print("\n🔮 You sense something...")
            for hint in hints:
                print(f"   {hint}")

    def play(self):
        print("🏰 Welcome to the Maze Escape Adventure! 🏰")
        print("\n📖 STORY:")
        print("You are trapped in a mysterious maze filled with dangers.")
        print("Navigate to the exit (🎯) while avoiding deadly traps!")
        print("\n🎮 CONTROLS:")
        print("N = North, S = South, E = East, W = West")
        print("Q = Quit game")
        print("\n🏠 ROOM TYPES:")
        print("⬜ = Safe room")
        print("🐍 = Snake (loses 1 health)")
        print("💣 = Bomb (instant death)")
        print("⭐ = Teleporter (random transport)")
        print("🎯 = Exit (victory!)")

        input("\nPress Enter to start your adventure...")

        # Mark starting position as visited
        self.visited.add(tuple(self.player_pos))

        while not self.game_over:
            self.clear_screen()
            self.print_maze()
            self.show_hints()

            # 🔁 Always handle the current room (unless already processed this turn)
            # If we just moved, the new cell is NOT yet in visited: trigger event.
            if tuple(self.player_pos) not in self.visited or self.maze[self.player_pos[0]][self.player_pos[1]] == self.TELEPORTER:
                print("\n" + "="*30)
                self.handle_room_event()
                # mark whatever tile we ended up on as visited
                self.visited.add(tuple(self.player_pos))

                if self.game_over:
                    break

            print("\n" + "="*30)
            move = input("🧭 Enter your move (N/S/E/W) or Q to quit: ").strip()

            if move.upper() == 'Q':
                print("👋 Thanks for playing! Goodbye!")
                break

            moved = self.move_player(move)
            # We do NOT mark visited here; event happens next loop.

        # Game over screen
        self.clear_screen()
        self.print_maze()

        if self.won:
            print("\n🎉" + "="*48 + "🎉")
            print("🏆           VICTORY ACHIEVED!           🏆")
            print("🎉" + "="*48 + "🎉")
            print("🌟 You have successfully escaped the maze!")
            print("🦸 You are truly a maze master!")
        else:
            print("\n💀" + "="*48 + "💀")
            print("☠️            GAME OVER!              ☠️")
            print("💀" + "="*48 + "💀")
            print("😢 Better luck next time, brave adventurer!")

        print(f"\n📊 FINAL STATS:")
        print(f"❤️  Final Health: {self.health}/3")
        print(f"🗺️  Rooms Explored: {len(self.visited)}/{self.size * self.size}")

def main():
    print("🏰 MAZE ESCAPE ADVENTURE GAME 🏰")
    print("="*50)

    while True:
        try:
            size = input("Choose maze size (5 for beginner, 10 for expert) [default: 5]: ").strip()
            if size == "":
                size = 5
            else:
                size = int(size)

            if size < 3:
                print("❌ Maze too small! Minimum size is 3x3.")
                continue
            elif size > 15:
                print("❌ Maze too large! Maximum size is 15x15.")
                continue

            break
        except ValueError:
            print("❌ Please enter a valid number!")

    game = MazeGame(size)
    game.play()

if __name__ == "__main__":
    main()


🏰 MAZE ESCAPE ADVENTURE GAME 🏰
Choose maze size (5 for beginner, 10 for expert) [default: 5]: 10
🏰 Welcome to the Maze Escape Adventure! 🏰

📖 STORY:
You are trapped in a mysterious maze filled with dangers.
Navigate to the exit (🎯) while avoiding deadly traps!

🎮 CONTROLS:
N = North, S = South, E = East, W = West
Q = Quit game

🏠 ROOM TYPES:
⬜ = Safe room
🐍 = Snake (loses 1 health)
💣 = Bomb (instant death)
⭐ = Teleporter (random transport)
🎯 = Exit (victory!)

Press Enter to start your adventure...Enter

🏰 MAZE ESCAPE ADVENTURE 🏰
  🧙 ❓ ❓ ❓ ❓ ❓ ❓ ❓ ⭐ ❓ 
  ❓ ❓ ❓ ❓ ❓ ❓ ❓ ⭐ ❓ ❓ 
  ❓ ❓ ❓ ❓ ⭐ ❓ ❓ ❓ ❓ ❓ 
  ❓ ⭐ ❓ ❓ ❓ ❓ ❓ ❓ ❓ ❓ 
  ❓ ⭐ ❓ ❓ ❓ ❓ ❓ ❓ ⭐ ❓ 
  ❓ ❓ ⭐ ❓ ❓ ❓ ⭐ ❓ ❓ ❓ 
  ❓ ❓ ❓ ❓ ⭐ ❓ ❓ ❓ ❓ ❓ 
  ❓ ❓ ❓ ⭐ ❓ ❓ ❓ ❓ ❓ ❓ 
  ❓ ❓ ⭐ ❓ ⭐ ❓ ❓ ❓ ❓ ❓ 
  ❓ ❓ ❓ ❓ ❓ ❓ ❓ ❓ ❓ 🎯 

❤️  Health: 3/3
📍 Position: (0, 0)
🎯 Exit: (9, 9)

🧭 Enter your move (N/S/E/W) or Q to quit: N
🚧 You hit a wall trying to go North! Choose another direction.

🏰 MAZE ESCAPE ADVENTURE 🏰
  🧙 ❓ ❓ ❓ ❓ ❓ ❓ ❓ ⭐ ❓ 
  ❓ ❓ ❓ ❓ ❓ ❓ ❓ ⭐ ❓ ❓ 
  

In [ ]:
import random
import os

class MazeGame:
    def __init__(self, size=5):
        self.size = size
        self.player_pos = [0, 0]                 # [row, col]
        self.exit_pos = [size-1, size-1]         # Bottom-right corner
        self.health = 3
        self.visited = set()
        self.game_over = False
        self.won = False

        # ⚠️ Room types
        self.EMPTY = '.'
        self.SNAKE = 'S'
        self.BOMB = 'B'
        self.TELEPORTER = 'Y'
        self.EXIT = 'X'
        self.PLAYER = 'P'

        self.maze = self.generate_maze()

    def generate_maze(self):
        """Generate a random maze with different room types"""
        maze = [[self.EMPTY for _ in range(self.size)] for _ in range(self.size)]

        # Place exit
        maze[self.exit_pos[0]][self.exit_pos[1]] = self.EXIT

        total_rooms = self.size * self.size - 2
        num_snakes = max(1, total_rooms // 4)       # ~25%
        num_bombs = max(1, total_rooms // 6)        # ~17%
        num_teleporters = max(1, total_rooms // 8)  # ~12%

        positions = [[r, c] for r in range(self.size) for c in range(self.size)
                     if [r, c] != self.player_pos and [r, c] != self.exit_pos]

        random.shuffle(positions)
        idx = 0

        for _ in range(min(num_snakes, len(positions))):
            r, c = positions[idx]; idx += 1
            maze[r][c] = self.SNAKE

        for _ in range(min(num_bombs, len(positions) - idx)):
            r, c = positions[idx]; idx += 1
            maze[r][c] = self.BOMB

        for _ in range(min(num_teleporters, len(positions) - idx)):
            r, c = positions[idx]; idx += 1
            maze[r][c] = self.TELEPORTER

        return maze

    def clear_screen(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    def print_maze(self):
        print("\n" + "="*50)
        print("🏰 MAZE ESCAPE ADVENTURE 🏰")
        print("="*50)

        for r in range(self.size):
            row_display = ""
            for c in range(self.size):
                if [r, c] == self.player_pos:
                    row_display += "🧙 "
                elif (r, c) in self.visited:
                    # Show true type for visited rooms
                    room_type = self.maze[r][c]
                    if room_type == self.EMPTY:
                        row_display += "⬜ "
                    elif room_type == self.SNAKE:
                        row_display += "🐍 "
                    elif room_type == self.BOMB:
                        row_display += "💣 "
                    elif room_type == self.TELEPORTER:
                        row_display += "✨ "
                    elif room_type == self.EXIT:
                        row_display += "🚪 "
                elif [r, c] == self.exit_pos:
                    row_display += "🎯 "
                elif self.maze[r][c] in (self.SNAKE, self.BOMB, self.TELEPORTER):
                    # ⭐ Special marker for hidden dangers
                    row_display += "⭐ "
                else:
                    row_display += "❓ "
            print(f"  {row_display}")

        print(f"\n❤️  Health: {self.health}/3")
        print(f"📍 Position: ({self.player_pos[0]}, {self.player_pos[1]})")
        print(f"🎯 Exit: ({self.exit_pos[0]}, {self.exit_pos[1]})")

    def get_room_hint(self, row, col):
        hints = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.size and 0 <= nc < self.size:
                room_type = self.maze[nr][nc]
                if room_type == self.SNAKE and (nr, nc) not in self.visited:
                    hints.append("🐍 You hear hissing nearby...")
                elif room_type == self.BOMB and (nr, nc) not in self.visited:
                    hints.append("💥 You smell gunpowder in the air...")
                elif room_type == self.TELEPORTER and (nr, nc) not in self.visited:
                    hints.append("✨ The air shimmers with magical energy...")
        return hints

    def handle_room_event(self):
        r, c = self.player_pos
        room_type = self.maze[r][c]

        if room_type == self.EMPTY:
            print("🟢 Safe room - You can rest here.")

        elif room_type == self.SNAKE:
            print("🐍 SNAKE ATTACK! A venomous snake bites you!")
            self.health -= 1
            print(f"💔 You lose 1 health. Health remaining: {self.health}")
            if self.health <= 0:
                print("💀 The snake's venom overwhelms you...")
                self.game_over = True

        elif room_type == self.BOMB:
            print("💣 BOOM! You stepped on a bomb!")
            print("💀 The explosion is instant and deadly...")
            self.health = 0
            self.game_over = True

        elif room_type == self.TELEPORTER:
            print("✨ MAGICAL TELEPORTER ACTIVATED!")
            print("🌀 Reality swirls around you as you're transported...")

            candidates = [[row, col] for row in range(self.size) for col in range(self.size)
                          if self.maze[row][col] != self.BOMB and [row, col] != self.exit_pos and [row, col] != self.player_pos]

            if candidates:
                new_pos = random.choice(candidates)
                self.player_pos = new_pos
                print(f"⚡ You materialize at position ({new_pos[0]}, {new_pos[1]})!")

        elif room_type == self.EXIT:
            print("🎉 CONGRATULATIONS! You found the exit!")
            print("🏆 You have successfully escaped the maze!")
            self.won = True
            self.game_over = True

    def move_player(self, direction):
        r, c = self.player_pos

        if direction.upper() == 'N':
            new_r, new_c = r - 1, c; dir_name = "North"
        elif direction.upper() == 'S':
            new_r, new_c = r + 1, c; dir_name = "South"
        elif direction.upper() == 'E':
            new_r, new_c = r, c + 1; dir_name = "East"
        elif direction.upper() == 'W':
            new_r, new_c = r, c - 1; dir_name = "West"
        else:
            print("❌ Invalid direction! Use N, S, E, or W")
            return False

        if new_r < 0 or new_r >= self.size or new_c < 0 or new_c >= self.size:
            print(f"🚧 You hit a wall trying to go {dir_name}! Choose another direction.")
            return False

        self.player_pos = [new_r, new_c]
        print(f"🚶 Moving {dir_name}...")
        return True

    def show_hints(self):
        hints = self.get_room_hint(self.player_pos[0], self.player_pos[1])
        if hints:
            print("\n🔮 You sense something...")
            for hint in hints:
                print(f"   {hint}")

    def play(self):
        print("🏰 Welcome to the Maze Escape Adventure! 🏰")
        print("\n📖 STORY:")
        print("You are trapped in a mysterious maze filled with dangers.")
        print("Navigate to the exit (🎯) while avoiding deadly traps!")
        print("\n🎮 CONTROLS:")
        print("N = North, S = South, E = East, W = West")
        print("Q = Quit game")
        print("\n🏠 ROOM TYPES:")
        print("⬜ = Safe room")
        print("⭐ = Mysterious room (could be Snake 🐍, Bomb 💣, or Teleporter ✨)")
        print("🎯 = Exit (victory!)")

        input("\nPress Enter to start your adventure...")

        self.visited.add(tuple(self.player_pos))

        while not self.game_over:
            self.clear_screen()
            self.print_maze()
            self.show_hints()

            if tuple(self.player_pos) not in self.visited or self.maze[self.player_pos[0]][self.player_pos[1]] == self.TELEPORTER:
                print("\n" + "="*30)
                self.handle_room_event()
                self.visited.add(tuple(self.player_pos))

                if self.game_over:
                    break

            print("\n" + "="*30)
            move = input("🧭 Enter your move (N/S/E/W) or Q to quit: ").strip()

            if move.upper() == 'Q':
                print("👋 Thanks for playing! Goodbye!")
                break

            self.move_player(move)

        self.clear_screen()
        self.print_maze()

        if self.won:
            print("\n🎉" + "="*48 + "🎉")
            print("🏆           VICTORY ACHIEVED!           🏆")
            print("🎉" + "="*48 + "🎉")
            print("🌟 You have successfully escaped the maze!")
            print("🦸 You are truly a maze master!")
        else:
            print("\n💀" + "="*48 + "💀")
            print("☠️            GAME OVER!              ☠️")
            print("💀" + "="*48 + "💀")
            print("😢 Better luck next time, brave adventurer!")

        print(f"\n📊 FINAL STATS:")
        print(f"❤️  Final Health: {self.health}/3")
        print(f"🗺️  Rooms Explored: {len(self.visited)}/{self.size * self.size}")


def main():
    print("🏰 MAZE ESCAPE ADVENTURE GAME 🏰")
    print("="*50)

    while True:
        try:
            size = input("Choose maze size (5 for beginner, 10 for expert) [default: 5]: ").strip()
            if size == "":
                size = 5
            else:
                size = int(size)

            if size < 3:
                print("❌ Maze too small! Minimum size is 3x3.")
                continue
            elif size > 15:
                print("❌ Maze too large! Maximum size is 15x15.")
                continue

            break
        except ValueError:
            print("❌ Please enter a valid number!")

    game = MazeGame(size)
    game.play()

if __name__ == "__main__":
    main()


🏰 MAZE ESCAPE ADVENTURE GAME 🏰
Choose maze size (5 for beginner, 10 for expert) [default: 5]: 10
🏰 Welcome to the Maze Escape Adventure! 🏰

📖 STORY:
You are trapped in a mysterious maze filled with dangers.
Navigate to the exit (🎯) while avoiding deadly traps!

🎮 CONTROLS:
N = North, S = South, E = East, W = West
Q = Quit game

🏠 ROOM TYPES:
⬜ = Safe room
⭐ = Mysterious room (could be Snake 🐍, Bomb 💣, or Teleporter ✨)
🎯 = Exit (victory!)

Press Enter to start your adventure...Enter

🏰 MAZE ESCAPE ADVENTURE 🏰
  🧙 ⭐ ⭐ ❓ ❓ ❓ ⭐ ❓ ❓ ⭐ 
  ❓ ❓ ⭐ ❓ ❓ ⭐ ❓ ❓ ❓ ⭐ 
  ⭐ ⭐ ❓ ❓ ⭐ ⭐ ❓ ❓ ⭐ ❓ 
  ❓ ⭐ ⭐ ❓ ❓ ⭐ ❓ ❓ ❓ ⭐ 
  ❓ ⭐ ⭐ ❓ ⭐ ⭐ ⭐ ⭐ ❓ ❓ 
  ❓ ❓ ❓ ⭐ ⭐ ❓ ❓ ⭐ ⭐ ❓ 
  ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ ❓ 
  ❓ ⭐ ⭐ ⭐ ⭐ ❓ ❓ ❓ ⭐ ⭐ 
  ❓ ❓ ⭐ ⭐ ⭐ ⭐ ⭐ ❓ ❓ ❓ 
  ❓ ⭐ ⭐ ⭐ ⭐ ⭐ ❓ ❓ ⭐ 🎯 

❤️  Health: 3/3
📍 Position: (0, 0)
🎯 Exit: (9, 9)

🔮 You sense something...
   🐍 You hear hissing nearby...

🧭 Enter your move (N/S/E/W) or Q to quit: S
🚶 Moving South...

🏰 MAZE ESCAPE ADVENTURE 🏰
  ⬜ ⭐ ⭐ ❓ ❓ ❓ ⭐ ❓ ❓ ⭐ 
  🧙 ❓ ⭐ ❓ ❓ ⭐ ❓ ❓ ❓ ⭐ 
  ⭐ ⭐ ❓ ❓ ⭐

In [ ]:
import random
import os

# Game constants
SIZE = 10
HEALTH = 3
DANGEROUS_SYMBOL = "⭐"
WALL = "⬛"
EMPTY = "⬜"
PLAYER = "🧑"
EXIT = "🚪"

# Dangerous room types
DANGERS = ["snake", "bomb", "teleport"]

def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")

def generate_maze(size):
    """Generate a random maze grid with walls, empty spaces, and dangers."""
    maze = [[EMPTY for _ in range(size)] for _ in range(size)]

    # Add random walls
    for r in range(size):
        for c in range(size):
            if random.random() < 0.2 and not (r == 0 and c == 0) and not (r == size - 1 and c == size - 1):
                maze[r][c] = WALL

    # Place dangers
    danger_positions = []
    for _ in range(size * 2):  # number of dangerous rooms
        r, c = random.randint(0, size - 1), random.randint(0, size - 1)
        if maze[r][c] == EMPTY and (r, c) not in [(0, 0), (size - 1, size - 1)]:
            maze[r][c] = DANGEROUS_SYMBOL
            danger_positions.append(((r, c), random.choice(DANGERS)))

    return maze, dict(danger_positions)

def render_maze(maze, player_pos, exit_pos):
    """Display the maze in the terminal."""
    clear_screen()
    for r in range(SIZE):
        row = ""
        for c in range(SIZE):
            if (r, c) == player_pos:
                row += PLAYER
            elif (r, c) == exit_pos:
                row += EXIT
            else:
                row += maze[r][c]
        print(row)

def move_player(pos, direction):
    """Move player based on WASD input."""
    r, c = pos
    if direction == "w":
        r -= 1
    elif direction == "s":
        r += 1
    elif direction == "a":
        c -= 1
    elif direction == "d":
        c += 1
    return r, c

def main():
    maze, dangers = generate_maze(SIZE)
    player_pos = (0, 0)
    exit_pos = (SIZE - 1, SIZE - 1)
    health = HEALTH

    while True:
        render_maze(maze, player_pos, exit_pos)
        print(f"\nHealth: {health}")
        move = input("Move (WASD): ").lower()

        if move not in ["w", "a", "s", "d"]:
            print("Invalid move! Use W A S D.")
            continue

        new_pos = move_player(player_pos, move)

        # Check boundaries
        r, c = new_pos
        if r < 0 or r >= SIZE or c < 0 or c >= SIZE:
            print("You hit the edge of the maze!")
            continue

        # Check wall
        if maze[r][c] == WALL:
            print("A wall blocks your way!")
            continue

        player_pos = new_pos

        # Check win
        if player_pos == exit_pos:
            render_maze(maze, player_pos, exit_pos)
            print("\n🎉 You escaped the maze safely! 🎉")
            break

        # Check dangers
        if player_pos in dangers:
            danger = dangers[player_pos]
            if danger == "snake":
                health -= 1
                print("🐍 A snake bit you! Health -1")
                if health <= 0:
                    print("💀 You died from snake bites!")
                    break
            elif danger == "bomb":
                print("💣 You stepped on a bomb! GAME OVER 💀")
                break
            elif danger == "teleport":
                print("🌀 A teleport room! You are warped to a random spot!")
                while True:
                    new_r, new_c = random.randint(0, SIZE - 1), random.randint(0, SIZE - 1)
                    if maze[new_r][new_c] != WALL:
                        player_pos = (new_r, new_c)
                        break

    print("\nGame Over!")

if __name__ == "__main__":
    main()
s


🧑⬛⬜⬛⬜⭐⬜⬜⬜⭐
⬜⬛⭐⬜⭐⬛⬜⬜⬜⬜
⭐⭐⬜⬜⬜⬜⬜⭐⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 3
Move (WASD): d
A wall blocks your way!
🧑⬛⬜⬛⬜⭐⬜⬜⬜⭐
⬜⬛⭐⬜⭐⬛⬜⬜⬜⬜
⭐⭐⬜⬜⬜⬜⬜⭐⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 3
Move (WASD): s
⬜⬛⬜⬛⬜⭐⬜⬜⬜⭐
🧑⬛⭐⬜⭐⬛⬜⬜⬜⬜
⭐⭐⬜⬜⬜⬜⬜⭐⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 3
Move (WASD): s
🐍 A snake bit you! Health -1
⬜⬛⬜⬛⬜⭐⬜⬜⬜⭐
⬜⬛⭐⬜⭐⬛⬜⬜⬜⬜
🧑⭐⬜⬜⬜⬜⬜⭐⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 2
Move (WASD): s
⬜⬛⬜⬛⬜⭐⬜⬜⬜⭐
⬜⬛⭐⬜⭐⬛⬜⬜⬜⬜
⭐⭐⬜⬜⬜⬜⬜⭐⬜⬜
🧑⬜⬛⬜⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 2
Move (WASD): d
⬜⬛⬜⬛⬜⭐⬜⬜⬜⭐
⬜⬛⭐⬜⭐⬛⬜⬜⬜⬜
⭐⭐⬜⬜⬜⬜⬜⭐⬜⬜
⬜🧑⬛⬜⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 2
Move (WASD): s
⬜⬛⬜⬛⬜⭐⬜⬜⬜⭐
⬜⬛⭐⬜⭐⬛⬜⬜⬜⬜
⭐⭐⬜⬜⬜⬜⬜⭐⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬛⬜
⬛🧑⬜⬛⬜⬜⭐⬜⬜⬜
⬛⭐⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⭐⬛⬜⬛⬜⬛
⭐⬜⬜⬜⬛⬛⬜⬜⬜⬛
⬜⭐⬜⭐⬛⬜⬜⬛⬛⬜
⭐⬜⬛⬜⭐⬜⭐⬜⬜🚪

Health: 2
Move (WASD

In [ ]:
import os
import random

# Game constants
SIZE = 10
HEALTH = 3
WALL = "⬛"
EMPTY = "⬜"
PLAYER = "🧑"
EXIT = "🚪"
DANGEROUS_SYMBOL = "⭐"

def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")

def fixed_maze():
    """Predefined 10x10 maze layout (no randomness)."""
    # Start with all empty
    maze = [[EMPTY for _ in range(SIZE)] for _ in range(SIZE)]

    # Add walls manually
    walls = [
        (0,3),(0,4),(0,6),(1,1),(1,6),(2,1),(2,5),(2,7),
        (3,3),(3,8),(4,0),(4,4),(5,4),(5,6),(6,2),(6,7),
        (7,3),(7,8),(8,5),(9,2),(9,6)
    ]
    for r,c in walls:
        maze[r][c] = WALL

    # Add dangers in fixed positions
    dangers = {
        (2,3): "snake",
        (7,5): "snake",
        (5,2): "bomb",
        (4,7): "teleport",
        (8,1): "teleport"
    }
    for pos in dangers:
        r,c = pos
        maze[r][c] = DANGEROUS_SYMBOL

    return maze, dangers

def render_maze(maze, player_pos, exit_pos):
    clear_screen()
    for r in range(SIZE):
        row = ""
        for c in range(SIZE):
            if (r,c) == player_pos:
                row += PLAYER
            elif (r,c) == exit_pos:
                row += EXIT
            else:
                row += maze[r][c]
        print(row)

def move_player(pos, direction):
    r, c = pos
    if direction == "w": r -= 1
    elif direction == "s": r += 1
    elif direction == "a": c -= 1
    elif direction == "d": c += 1
    return r, c

def main():
    maze, dangers = fixed_maze()
    player_pos = (0,0)
    exit_pos = (SIZE-1,SIZE-1)
    health = HEALTH

    while True:
        render_maze(maze, player_pos, exit_pos)
        print(f"\nHealth: {health}")
        move = input("Move (WASD): ").lower()

        if move not in ["w","a","s","d"]:
            print("Invalid move!")
            continue

        new_pos = move_player(player_pos, move)
        r,c = new_pos

        # Boundaries
        if r < 0 or r >= SIZE or c < 0 or c >= SIZE:
            print("You hit the edge of the maze!")
            continue

        # Wall
        if maze[r][c] == WALL:
            print("A wall blocks your path!")
            continue

        player_pos = new_pos

        # Win check
        if player_pos == exit_pos:
            render_maze(maze, player_pos, exit_pos)
            print("\n🎉 You escaped the maze safely! 🎉")
            break

        # Danger check
        if player_pos in dangers:
            danger = dangers[player_pos]
            if danger == "snake":
                health -= 1
                print("🐍 A snake bit you! Health -1")
                if health <= 0:
                    print("💀 You died from snake bites!")
                    break
            elif danger == "bomb":
                print("💣 You stepped on a bomb! GAME OVER 💀")
                break
            elif danger == "teleport":
                print("🌀 Teleport trap! You are warped to a random safe spot.")
                while True:
                    new_r,new_c = random.randint(0,SIZE-1), random.randint(0,SIZE-1)
                    if maze[new_r][new_c] != WALL:
                        player_pos = (new_r,new_c)
                        break

    print("\nGame Over!")

if __name__ == "__main__":
    main()



🧑⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): D
⬜🧑⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): D
⬜⬜🧑⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): s
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛🧑⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): s
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛🧑⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): w
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛🧑⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): d
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜🧑⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): d
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜🧑⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜
⬛⬜⬜⬜

In [ ]:
import os
import random

# Game constants
SIZE = 20
HEALTH = 3
WALL = "⬛"
EMPTY = "⬜"
PLAYER = "🧑"
EXIT = "🚪"
DANGEROUS_SYMBOL = "⭐"

def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")

def fixed_maze():
    """Predefined 20x20 maze layout with 10 dangers."""
    maze = [[EMPTY for _ in range(SIZE)] for _ in range(SIZE)]

    # Add walls (handmade pattern, not blocking the exit completely)
    walls = []
    for r in range(SIZE):
        for c in range(SIZE):
            if (r % 2 == 0 and c % 4 == 0) or (c % 2 == 0 and r % 5 == 0):
                if (r, c) not in [(0,0), (SIZE-1, SIZE-1)]:
                    walls.append((r, c))
    for r,c in walls:
        maze[r][c] = WALL

    # Add exactly 10 dangers in fixed positions
    dangers = {
        (2,3): "snake",
        (4,10): "snake",
        (6,15): "bomb",
        (8,5): "teleport",
        (9,13): "teleport",
        (12,7): "snake",
        (14,16): "bomb",
        (15,4): "snake",
        (17,12): "teleport",
        (18,18): "snake"
    }
    for pos in dangers:
        r,c = pos
        maze[r][c] = DANGEROUS_SYMBOL

    return maze, dangers

def render_maze(maze, player_pos, exit_pos):
    clear_screen()
    for r in range(SIZE):
        row = ""
        for c in range(SIZE):
            if (r,c) == player_pos:
                row += PLAYER
            elif (r,c) == exit_pos:
                row += EXIT
            else:
                row += maze[r][c]
        print(row)

def move_player(pos, direction):
    r, c = pos
    if direction == "w": r -= 1
    elif direction == "s": r += 1
    elif direction == "a": c -= 1
    elif direction == "d": c += 1
    return r, c

def main():
    maze, dangers = fixed_maze()
    player_pos = (0,0)
    exit_pos = (SIZE-1,SIZE-1)
    health = HEALTH

    while True:
        render_maze(maze, player_pos, exit_pos)
        print(f"\nHealth: {health}")
        move = input("Move (WASD): ").lower()

        if move not in ["w","a","s","d"]:
            print("Invalid move!")
            continue

        new_pos = move_player(player_pos, move)
        r,c = new_pos

        # Boundaries
        if r < 0 or r >= SIZE or c < 0 or c >= SIZE:
            print("You hit the edge of the maze!")
            continue

        # Wall
        if maze[r][c] == WALL:
            print("A wall blocks your path!")
            continue

        player_pos = new_pos

        # Win check
        if player_pos == exit_pos:
            render_maze(maze, player_pos, exit_pos)
            print("\n🎉 You escaped the 20x20 maze safely! 🎉")
            break

        # Danger check
        if player_pos in dangers:
            danger = dangers[player_pos]
            if danger == "snake":
                health -= 1
                print("🐍 A snake bit you! Health -1")
                if health <= 0:
                    print("💀 You died from snake bites!")
                    break
            elif danger == "bomb":
                print("💣 You stepped on a bomb! GAME OVER 💀")
                break
            elif danger == "teleport":
                print("🌀 Teleport trap! You are warped to a random safe spot.")
                while True:
                    new_r,new_c = random.randint(0,SIZE-1), random.randint(0,SIZE-1)
                    if maze[new_r][new_c] != WALL:
                        player_pos = (new_r,new_c)
                        break

    print("\nGame Over!")

if __name__ == "__main__":
    main()


🧑⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): s
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
🧑⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): d
A wall blocks your path!
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
🧑⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): d
A wall blocks your path!
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
🧑⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): 
Invalid move!
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
🧑⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): s
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
🧑⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜⬜🚪

Health: 3
Move (WASD): d
A wall blocks your path!
⬜⬜⬜⬛⬛⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
🧑⬛⬜⭐⬜⬛⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⭐⬜⬛⬜
⬛⬜⬜⬜⬛⬜⬜⭐⬜⬜
⬜⬜⭐⬜⬛⬜⬛⬜⬜⬜
⬜⬜⬛⬜⭐⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⭐⬜⬜⬛⬜
⬜⭐⬜⬜⬜⬛⬜⬜

In [ ]:
import os
import random

# Game constants
SIZE = 10
HEALTH = 3
WALL = "⬛"
EMPTY = "⬜"
PLAYER = "🧑"
EXIT = "🚪"
UNKNOWN = "⭐"  # Unknown danger

# Real danger symbols
SYMBOLS = {
    "snake": "🐍",
    "bomb": "💣",
    "teleport": "🌀",
    "heal": "❤️"
}

def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")

def show_rules():
    clear_screen()
    print("🎮 Maze Escape Rules 🎮\n")
    print("⬛ = Wall (you cannot move through it)")
    print("⬜ = Empty room")
    print("🧑 = You (the player)")
    print("🚪 = Exit (goal)")
    print("⭐ = Unknown danger room (one of the following):")
    print("   🐍 Snake → Lose 1 health")
    print("   💣 Bomb → Instant Game Over")
    print("   🌀 Teleport → Randomly move to another room")
    print("   ❤️ Heal → Restore full health\n")
    print("Health starts at 3. Reach the exit to win!\n")
    input("Press ENTER to start...")

def fixed_maze():
    """Predefined 10x10 maze layout with 7 fixed danger rooms."""
    # Start with all empty
    maze = [[EMPTY for _ in range(SIZE)] for _ in range(SIZE)]

    # Add walls manually
    walls = [
        (0,3),(0,4),(0,6),(1,1),(1,6),(2,1),(2,5),(2,7),
        (3,3),(3,8),(4,0),(4,4),(5,4),(5,6),(6,2),(6,7),
        (7,3),(7,8),(8,5),(9,2),(9,6)
    ]
    for r,c in walls:
        maze[r][c] = WALL

    # Add dangers in fixed positions
    dangers = {
        (2,3): "snake",
        (7,5): "snake",
        (5,2): "bomb",
        (4,7): "teleport",
        (8,1): "teleport",
        (6,4): "teleport",
        (3,6): "heal"
    }

    # At first, mark them as ⭐
    for pos in dangers:
        r,c = pos
        maze[r][c] = UNKNOWN

    return maze, dangers

def render_maze(maze, player_pos, exit_pos):
    clear_screen()
    for r in range(SIZE):
        row = ""
        for c in range(SIZE):
            if (r,c) == player_pos:
                row += PLAYER
            elif (r,c) == exit_pos:
                row += EXIT
            else:
                row += maze[r][c]
        print(row)

def move_player(pos, direction):
    r, c = pos
    if direction == "w": r -= 1
    elif direction == "s": r += 1
    elif direction == "a": c -= 1
    elif direction == "d": c += 1
    return r, c

def main():
    show_rules()
    maze, dangers = fixed_maze()
    player_pos = (0,0)
    exit_pos = (SIZE-1,SIZE-1)
    health = HEALTH

    revealed = {}  # stores revealed dangers after visiting

    while True:
        render_maze(maze, player_pos, exit_pos)
        print(f"\nHealth: {health}")
        move = input("Move (WASD): ").lower()

        if move not in ["w","a","s","d"]:
            print("Invalid move!")
            continue

        new_pos = move_player(player_pos, move)
        r,c = new_pos

        # Boundaries
        if r < 0 or r >= SIZE or c < 0 or c >= SIZE:
            print("You hit the edge of the maze!")
            continue

        # Wall
        if maze[r][c] == WALL:
            print("A wall blocks your path!")
            continue

        player_pos = new_pos

        # Win check
        if player_pos == exit_pos:
            render_maze(maze, player_pos, exit_pos)
            print("\n🎉 You escaped the maze safely! 🎉")
            break

        # Danger check
        if player_pos in dangers:
            danger = dangers[player_pos]

            # Reveal the danger permanently
            maze[r][c] = SYMBOLS[danger]
            revealed[player_pos] = danger

            if danger == "snake":
                health -= 1
                print("🐍 A snake bit you! Health -1")
                if health <= 0:
                    print("💀 You died from snake bites!")
                    break
            elif danger == "bomb":
                print("💣 You stepped on a bomb! GAME OVER 💀")
                break
            elif danger == "teleport":
                print("🌀 Teleport trap! You are warped to a random safe spot.")
                while True:
                    new_r,new_c = random.randint(0,SIZE-1), random.randint(0,SIZE-1)
                    if maze[new_r][new_c] != WALL:
                        player_pos = (new_r,new_c)
                        break
            elif danger == "heal":
                health = HEALTH
                print("❤️ You found a healing fountain! Health restored to full.")

    print("\nGame Over!")

if __name__ == "__main__":
    main()


🎮 Maze Escape Rules 🎮

⬛ = Wall (you cannot move through it)
⬜ = Empty room
🧑 = You (the player)
🚪 = Exit (goal)
⭐ = Unknown danger room (one of the following):
   🐍 Snake → Lose 1 health
   💣 Bomb → Instant Game Over
   🌀 Teleport → Randomly move to another room
   ❤️ Heal → Restore full health

Health starts at 3. Reach the exit to win!

